In [110]:
print("Ello govna") #british for hello#

print("Oi tudo bem?") #portuguese for how are you?#

print("Tuxedo") #american for fancy clothes#

print("Terno") #portuguese for tuxedo#

Ello govna
Oi tudo bem?
Tuxedo
Terno
time: 0 ns (started: 2023-12-15 11:06:39 -05:00)


In [111]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2023-12-15 11:06:39 -05:00)


In [112]:
file_path = r"C:\Users\aaron\Documents\Python Code\Datasets\Heart Failure Prediction\heart.csv"

heart_attack = pd.read_csv(file_path)

heart_attack.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


time: 0 ns (started: 2023-12-15 11:06:39 -05:00)


In [113]:
#Defining mapping function to map multiple inputs of the dataset
def gender_map(x):
    return x.map({'M': 1, 'F': 0, 'Male' : 1, "Female" : 0, '1' : 1, '0' : 0})

def hemisphere_map(x):
    return x.map({'Southern Hemisphere' : -1, 'Northern Hemisphere' : 1})

def countries_map(x):
    return x.map({'Argentina' : -1, 'Brazil' : -1, 'China' : -1, 'Colombia' : -1,
    'India' : -1, 'Nigeria' : -1, 'South Africa' : -1, 'South Korea' : -1, 'Thailand' : -1,
    'Vietnam' : -1, 'Australia' : 1, 'Canada' : 1, 'France' : 1, 'Germany' : 1,
    'Italy' : 1, 'Japan' : 1, 'New Zealand' : 1, 'Spain' : 1, 'United Kingdom' : 1,
    'United States' : 1})

def diet_map(x):
    return x.map({'Unhealthy' : -1, 'Average' : 0, 'Healthy' : 1})

def continent_map(x):
    return x.map({'Asia' : 0, 'Africa' : 1, 'Europe' : 2, 'North America' : 3,
    'South America' : 4, 'Australia' : 5})

def ExerciseAngina_map(x):
    return x.map({'Y' : 1, 'N' : 0})

def ST_Slope_map(x):
    return x.map({'Down' : -1, 'Flat' : 0, 'Up' : 1})


def ChestPainType_map(x):
    return x.map({'ATA' : 1, 'NAP' : 0.5, 'ASY' : -0.5, 'TA' : -1})

def RestingECG_map(x):
    return x.map({'LVH' : -1, 'Normal' : 0, 'ST' : 1})


time: 0 ns (started: 2023-12-15 11:06:39 -05:00)


In [114]:
heart_attack['Sex'] = heart_attack[['Sex']].apply(gender_map)
heart_attack['ExerciseAngina'] = heart_attack[['ExerciseAngina']].apply(ExerciseAngina_map)
heart_attack['ST_Slope'] = heart_attack[['ST_Slope']].apply(ST_Slope_map)
heart_attack['ChestPainType'] = heart_attack[['ChestPainType']].apply(ChestPainType_map)
heart_attack['RestingECG'] = heart_attack[['RestingECG']].apply(RestingECG_map)
# heart_attack['Hemisphere'] = heart_attack[['Hemisphere']].apply(hemisphere_map)
# heart_attack['Country'] = heart_attack[['Country']].apply(countries_map)
# heart_attack['Diet'] = heart_attack[['Diet']].apply(diet_map)
# heart_attack['Continent'] = heart_attack[['Continent']].apply(continent_map)

unique_sex = heart_attack['Sex'].unique()
print(unique_sex)
# unique = heart_attack['Hemisphere'].unique()
# print(unique)
# unique_country = heart_attack['Country'].unique()
# print(unique_country)
# unique_diet = heart_attack['Diet'].unique()
# print(unique_diet)
# unique_continent = heart_attack['Continent'].unique()
# print(unique_continent)

heart_attack.head()

[1 0]


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1.0,140,289,0,0,172,0,0.0,1,0
1,49,0,0.5,160,180,0,0,156,0,1.0,0,1
2,37,1,1.0,130,283,0,1,98,0,0.0,1,0
3,48,0,-0.5,138,214,0,0,108,1,1.5,0,1
4,54,1,0.5,150,195,0,0,122,0,0.0,1,0


time: 0 ns (started: 2023-12-15 11:06:39 -05:00)


In [115]:
# mode_columns = ['Stress Level','Medication Use', 'Previous Heart Problems', 'Diet', 'Physical Activity Days Per Week', 'Alcohol Consumption', 'Obesity', 'Smoking', 
#                 'Family History', 'Diabetes', 'Hemisphere', 'Continent', 'Country']

# mean_columns = ['Exercise Hours Per Week', 'Sleep Hours Per Day', 'BMI', 'Sedentary Hours Per Day']

# median_columns = ['Heart Rate', 'Income', 'Triglycerides']

# # Fill NaN values with the mode of the column for mode_columns
# for col in mode_columns:
#     heart_attack[col].fillna(heart_attack[col].mode()[0], inplace=True)

# # Fill NaN values with the median of the column for median_columns
# for col in median_columns:
#     heart_attack[col].fillna(heart_attack[col].median(), inplace=True)

# # Fill NaN values with the mean of the column for mean_columns
# for col in mean_columns:
#     heart_attack[col].fillna(heart_attack[col].mean(), inplace=True)
    

# unique_sex = heart_attack['Sex'].unique()
# print(unique_sex)
# unique = heart_attack['Hemisphere'].unique()
# print(unique)
# unique_country = heart_attack['Country'].unique()
# print(unique_country)
# unique_diet = heart_attack['Diet'].unique()
# print(unique_diet)
# unique_continent = heart_attack['Continent'].unique()
# print(unique_continent)


# Check for NaN values in columns after fillna operations
columns_with_nulls = heart_attack.columns[heart_attack.isnull().any()].tolist()
print("Columns with NaN values after fillna operations:")
print(columns_with_nulls)

heart_attack.head()



Columns with NaN values after fillna operations:
[]


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1.0,140,289,0,0,172,0,0.0,1,0
1,49,0,0.5,160,180,0,0,156,0,1.0,0,1
2,37,1,1.0,130,283,0,1,98,0,0.0,1,0
3,48,0,-0.5,138,214,0,0,108,1,1.5,0,1
4,54,1,0.5,150,195,0,0,122,0,0.0,1,0


time: 16 ms (started: 2023-12-15 11:06:39 -05:00)


In [116]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU
    print("GPU is available.")
else:
    device = torch.device("cpu")   # Use CPU
    print("GPU is not available, using CPU.")

GPU is not available, using CPU.
time: 0 ns (started: 2023-12-15 11:06:39 -05:00)


In [117]:
#Splitting the Variables into X and Y
X = heart_attack.iloc[:, 0:10].values
Y = heart_attack.iloc[:, 11].values

# Split our Data set into Training Data and val Data.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# Use standard scaling from Sklearn to scale data between - and  for better accuracy.
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_val = scale.transform(X_val)

Y_train = Y_train.reshape(-1, 1)
Y_val = Y_val.reshape(-1, 1)

X_train_t = torch.tensor(X_train, dtype=torch.float)
X_val_t = torch.tensor(X_val, dtype=torch.float)
Y_train_t = torch.tensor(Y_train, dtype=torch.float)
Y_val_t = torch.tensor(Y_val, dtype=torch.float)


print(X_train_t.dtype)
print(X_val_t.dtype)
print(Y_train_t.dtype)
print(Y_val_t.dtype)

torch.float32
torch.float32
torch.float32
torch.float32
time: 16 ms (started: 2023-12-15 11:06:39 -05:00)


In [118]:
def model_linear(t_in, w1, w2, w3, b):
  t_p = ((w3 * t_in ** 3) + (w2 * t_in ** 2) + (w1 * t_in) + b)
  return t_p

def loss_fn(t_p, t_gnd):
  squared_diffs = (t_p - t_gnd)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_in, val_t_in,
                    train_t_out, val_t_out):
  for epoch in range(1, n_epochs + 1):

    if params.grad is not None:
      params.grad.zero_()

    train_t_p = model_linear(train_t_in, *params)
    train_loss = loss_fn(train_t_p, train_t_out)

    val_t_p = model_linear(val_t_in, *params)
    val_loss = loss_fn(val_t_p, val_t_out)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, \tTraining loss {train_loss.item():.4f},"
      f" \tValidation loss {val_loss.item():.4f}")

  return params

time: 0 ns (started: 2023-12-15 11:06:39 -05:00)


In [119]:
params = torch.tensor([1.0, 1.0, 1.0, 0.0], requires_grad=True)

learning_rate_1 = 1e-3
optimizer_1 = optim.SGD([params], lr=learning_rate_1)

print('Loss every 500 epochs using learning rate of 1e-3\n')
training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1,
    params = params,
    train_t_in = X_train_t,
    val_t_in = X_val_t,
    train_t_out = Y_train_t,
    val_t_out = Y_val_t)

Loss every 500 epochs using learning rate of 1e-3

Epoch 1, 	Training loss 40.0893, 	Validation loss 27.2035
Epoch 2, 	Training loss 35.3552, 	Validation loss 24.5369
Epoch 3, 	Training loss 31.2246, 	Validation loss 22.1779
Epoch 500, 	Training loss 0.4926, 	Validation loss 0.4835
Epoch 1000, 	Training loss 0.3049, 	Validation loss 0.3080
Epoch 1500, 	Training loss 0.2603, 	Validation loss 0.2623
Epoch 2000, 	Training loss 0.2493, 	Validation loss 0.2494
Epoch 2500, 	Training loss 0.2466, 	Validation loss 0.2453
Epoch 3000, 	Training loss 0.2459, 	Validation loss 0.2438
Epoch 3500, 	Training loss 0.2457, 	Validation loss 0.2432
Epoch 4000, 	Training loss 0.2457, 	Validation loss 0.2429
Epoch 4500, 	Training loss 0.2457, 	Validation loss 0.2428
Epoch 5000, 	Training loss 0.2456, 	Validation loss 0.2427


tensor([ 3.8683e-02,  1.8843e-02, -4.5924e-04,  5.2654e-01],
       requires_grad=True)

time: 1.7 s (started: 2023-12-15 11:06:39 -05:00)


In [120]:
# Getting the predictions based on the training set using the linear model
model_output = model_linear(X_train_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_np = model_output.detach().numpy() if isinstance(model_output, torch.Tensor) else model_output

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels = np.argmax(model_output_np, axis=1)

# Calculate accuracy, precision, and recall for training set
train_accuracy = accuracy_score(Y_train, predicted_labels)
train_recall = recall_score(Y_train, predicted_labels, average= 'macro', zero_division=1)
train_precision = precision_score(Y_train, predicted_labels, average = 'macro', zero_division=1)
train_F1_score = (2*train_recall*train_precision)/(train_recall+train_precision)

# Getting the predictions based on the validation set using the linear model
model_output_val = model_linear(X_val_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_val_np = model_output_val.detach().numpy() if isinstance(model_output_val, torch.Tensor) else model_output_val

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels_val = np.argmax(model_output_val_np, axis=1)

# Calculate accuracy, precision, and recall for validation set
val_accuracy = accuracy_score(Y_val, predicted_labels_val)
val_precision = precision_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_recall = recall_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_F1_score = (2*val_recall*val_precision)/(val_recall+val_precision)

# Print the calculated metrics
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training Precision: {train_precision:.4f}")
print(f"Training Recall: {train_recall:.4f}")
print(f"Training F1 Score: {train_F1_score:.4f}")

print(f"\nValidation Accuracy: {val_accuracy:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"Validation F1 Score: {val_F1_score:.4f}")


Training Accuracy: 0.0422
Training Precision: 0.0860
Training Recall: 0.8089
Training F1 Score: 0.1554

Validation Accuracy: 0.0435
Validation Precision: 0.0829
Validation Recall: 0.8097
Validation F1 Score: 0.1503
time: 0 ns (started: 2023-12-15 11:06:41 -05:00)


In [121]:
#Sequential Model

model_seq = nn.Sequential(
            nn.Linear(10, 1024),
            nn.Tanh(),
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Linear(2048, 512),
            nn.Tanh(),
            nn.Linear(512,1)
            )
model_seq

Sequential(
  (0): Linear(in_features=10, out_features=1024, bias=True)
  (1): Tanh()
  (2): Linear(in_features=1024, out_features=2048, bias=True)
  (3): Tanh()
  (4): Linear(in_features=2048, out_features=512, bias=True)
  (5): Tanh()
  (6): Linear(in_features=512, out_features=1, bias=True)
)

time: 32 ms (started: 2023-12-15 11:06:41 -05:00)


In [122]:
# Checking the complexity of the model
sum([p.numel() for p in model_seq.parameters()])

3160065

time: 0 ns (started: 2023-12-15 11:06:41 -05:00)


In [123]:
#Let's define our training loop that will be used

def training_loop(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(x_train) # <1>
        loss_train = loss_fn(p_train, y_train,)

        p_val = model(x_val) # <1>
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()


        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch: {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}")

time: 0 ns (started: 2023-12-15 11:06:41 -05:00)


In [124]:
#Let's try doing some training

optimizer_1 = optim.SGD(model_seq.parameters(), lr=1e-2)

training_loop(
    n_epochs = 501,
    optimizer = optimizer_1,
    model = model_seq,
    loss_fn = nn.MSELoss(),
    x_train = X_train_t,
    x_val = X_val_t,
    y_train = Y_train_t,
    y_val = Y_val_t
)

Epoch: 1, Training loss: 0.5687, Validation loss: 0.6007
Epoch: 10, Training loss: 0.1387, Validation loss: 0.1567
Epoch: 20, Training loss: 0.1293, Validation loss: 0.1453
Epoch: 30, Training loss: 0.1288, Validation loss: 0.1454
Epoch: 40, Training loss: 0.1285, Validation loss: 0.1455
Epoch: 50, Training loss: 0.1283, Validation loss: 0.1455
Epoch: 60, Training loss: 0.1281, Validation loss: 0.1455
Epoch: 70, Training loss: 0.1279, Validation loss: 0.1454
Epoch: 80, Training loss: 0.1277, Validation loss: 0.1453
Epoch: 90, Training loss: 0.1274, Validation loss: 0.1452
Epoch: 100, Training loss: 0.1272, Validation loss: 0.1451
Epoch: 110, Training loss: 0.1270, Validation loss: 0.1450
Epoch: 120, Training loss: 0.1268, Validation loss: 0.1449
Epoch: 130, Training loss: 0.1266, Validation loss: 0.1449
Epoch: 140, Training loss: 0.1264, Validation loss: 0.1448
Epoch: 150, Training loss: 0.1262, Validation loss: 0.1447
Epoch: 160, Training loss: 0.1260, Validation loss: 0.1446
Epoch: 1

In [125]:
# Function to calculate accuracy
def calculate_accuracy(predictions, targets):
    return (predictions == targets).float().mean().item()

# Function to calculate recall
def calculate_recall(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_negatives = ((predictions == 0) & (targets == 1)).sum().item()
    return true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# Function to calculate precision
def calculate_precision(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_positive = ((predictions == 1) & (targets == 0)).sum().item()
    return true_positives / (true_positives + false_positive) if (true_positives + false_positive) > 0 else 0

time: 0 ns (started: 2023-12-15 11:07:18 -05:00)


In [126]:
# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_train_t)))

# Calculate accuracy, recall, precision, and F1 score for the training set
accuracy_train = calculate_accuracy(train_predictions, Y_train_t)
recall_train = calculate_recall(train_predictions, Y_train_t)
precision_train = calculate_precision(train_predictions, Y_train_t)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_val_t)))

# Calculate accuracy, recall, precision, and F1 score for the validation set
accuracy_val = calculate_accuracy(train_predictions, Y_val_t)
recall_val = calculate_recall(train_predictions, Y_val_t)
precision_val = calculate_precision(train_predictions, Y_val_t)
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)


# Print the metrics
print("Training Accuracy: {:.4f}".format(accuracy_train))
print("Training Recall: {:.4f}".format(recall_train))
print("Training Precision: {:.4f}".format(precision_train))
print("Training F1 Score: {:.4f}".format(F1_score_train))

print("\nValidation Accuracy: {:.4f}".format(accuracy_val))
print("Validation Recall: {:.4f}".format(recall_val))
print("Validation Precision: {:.4f}".format(precision_val))
print("Validation F1 Score: {:.4f}".format(F1_score_val))

Training Accuracy: 0.6022
Training Recall: 1.0000
Training Precision: 0.5786
Training F1 Score: 0.7331

Validation Accuracy: 0.6196
Validation Recall: 0.9907
Validation Precision: 0.6057
Validation F1 Score: 0.7518
time: 63 ms (started: 2023-12-15 11:07:18 -05:00)


In [127]:
# Defining the fully connected neural network

model_fnn = nn.Sequential(
            nn.Linear(10, 1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(2048,1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(512, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
           )

time: 31 ms (started: 2023-12-15 11:07:18 -05:00)


In [128]:
# Checking the complexity of the model
sum([p.numel() for p in model_fnn.parameters()])

4766337

time: 0 ns (started: 2023-12-15 11:07:18 -05:00)


In [129]:
# Setting up the parameters used in the training loop

learning_rate = 1

optimizer = optim.SGD(model_fnn.parameters(), lr=learning_rate)

loss_fn = nn.BCEWithLogitsLoss()

n_epochs = 501

# This is the Training

for epoch in range(n_epochs):
    outputs = model_fnn(X_train_t)
    loss = loss_fn(outputs, Y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # Print loss every 10 epochs
      print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.692373
Epoch: 10, Loss: 0.392574
Epoch: 20, Loss: 0.386617
Epoch: 30, Loss: 0.392131
Epoch: 40, Loss: 0.387376
Epoch: 50, Loss: 0.394274
Epoch: 60, Loss: 0.377916
Epoch: 70, Loss: 0.371168
Epoch: 80, Loss: 0.369393
Epoch: 90, Loss: 0.364172
Epoch: 100, Loss: 0.360983
Epoch: 110, Loss: 0.405952
Epoch: 120, Loss: 0.350617
Epoch: 130, Loss: 0.338590
Epoch: 140, Loss: 0.338950
Epoch: 150, Loss: 0.335121
Epoch: 160, Loss: 0.396078
Epoch: 170, Loss: 0.322670
Epoch: 180, Loss: 0.331197
Epoch: 190, Loss: 0.321049
Epoch: 200, Loss: 0.327126
Epoch: 210, Loss: 0.321410
Epoch: 220, Loss: 0.302293
Epoch: 230, Loss: 0.305238
Epoch: 240, Loss: 0.307207
Epoch: 250, Loss: 0.319675
Epoch: 260, Loss: 0.304992
Epoch: 270, Loss: 0.283882
Epoch: 280, Loss: 0.287652
Epoch: 290, Loss: 0.284337
Epoch: 300, Loss: 0.274885
Epoch: 310, Loss: 0.269028
Epoch: 320, Loss: 0.278816
Epoch: 330, Loss: 0.262231
Epoch: 340, Loss: 0.250628
Epoch: 350, Loss: 0.254388
Epoch: 360, Loss: 0.232419
Epoch: 370, 

In [130]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Training Set

correct_train = 0
total_train = 0

with torch.no_grad():
    outputs_train = model_fnn(X_train_t)
    _, predicted_train = torch.max(outputs_train, dim=1)
    total_train += Y_train_t.size(0)
    correct_train += int((predicted_train == Y_train_t.squeeze()).sum())

accuracy_train = correct_train / total_train
precision_train = precision_score(Y_train_t, predicted_train, average='macro', zero_division=0)
recall_train = recall_score(Y_train_t, predicted_train, average='macro', zero_division=0)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Print the metrics
print("Training Accuracy: %f" % accuracy_train)
print("Training Precision: %f" % precision_train)
print("Training Recall: %f" % recall_train)
print("Training F1 score: %f" % F1_score_train)

Training Accuracy: 0.453678
Training Precision: 0.226839
Training Recall: 0.500000
Training F1 score: 0.312090
time: 31 ms (started: 2023-12-15 11:08:09 -05:00)


In [131]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Validation Set

correct_val = 0
total_val = 0

with torch.no_grad():
    outputs_val = model_fnn(X_val_t)
    _, predicted_val = torch.max(outputs_val, dim=1)
    total_val += Y_val_t.size(0)
    correct_val += int((predicted_val == Y_val_t.squeeze()).sum())

accuracy_val = correct_val / total_val
precision_val = precision_score(Y_val_t, predicted_val, average='macro', zero_division=0)
pecall_val = recall_score(Y_val_t, predicted_val, average='macro', zero_division=0)
p1_score_train = (2*recall_val*precision_val)/(recall_val+precision_val)

# Print the metrics
print("Validation Accuracy: %f" % accuracy_val)
print("Validation Precision: %f" % precision_val)
print("Validation Recall: %f" % recall_val)
print("Validation F1 score: %f" % F1_score_train)

Validation Accuracy: 0.418478
Validation Precision: 0.209239
Validation Recall: 0.990654
Validation F1 score: 0.312090
time: 15 ms (started: 2023-12-15 11:08:09 -05:00)
